# Advanced inheritance

**Outline**:
- Reminder on inheritance
- Proper multiple inheritance
- Metaclasses
- Composition over inheritance
- Dynamic object creation

TODO:
- `super(X, y)` 
- `__init_subclass__`
- 


## Reminder on inheritance

### Inheritance basics
- ABC/ABCMeta, abstractmethod, super

### Multiple inheritance
- Diamond problem

## Proper multiple inheritance

### Pitfalls
The MRO ensures a clear ordering of the class hierarchy, with good properties and deterministic behavior. It does not solve all the problem however. Consider the following piece of code.

In [ ]:
class Printer:
    def output(self):
        return "Printing..."

class Logger:
    def output(self):
        return "Logging..."

class Service(Printer, Logger):
    pass

s = Service()
print(s.output())


In this case, we have a collision between the `output` methods, and one masks the other. This happens with common names, in both methods and attributes. A special case is dunder methods (where the name is imposed), with the most frequent issue being  `__init__`.

In [ ]:
class A:
    def __init__(self):
        print("Initializing A")

class B:
    def __init__(self):
        print("Initializing B")
        
class C(A, B):
    def __init__(self):
        super().__init__()
        print("Initializing C")

C()  # Skipping B

Note that you can partially circumvent the above issue with more specific calls, but this then breaks the MRO properties, leading to other issues

In [ ]:
class A:
    def __init__(self):
        print("Initializing A")

class B(A):
    def __init__(self):
        print("Initializing B")
        super().__init__()

class C(A):
    def __init__(self):
        print("Initializing C")
        super().__init__()

class D(B, C):
    def __init__(self):
        print("Initializing D")
        B.__init__(self) # Explicit choice of super class
        C.__init__(self) # Explicit choice of super class 

_ = D()  # C and A are initialized twice

The double initilization might just be a waste of time, but it might also be an issue in case of side effects.

The remainder of this section is about best practices in the context of multiple inheritance.

### Interface/protocol

In OOP, a common way of doing "soft" multiple inheritance is to have *interfaces*. An interface is like an API, it exposes some method that must be implemented but does not provide an implementation. It is a contract to the user saying that a concrete class will have this/those methods implemented. Typically, an object can implement several interfaces without risk name collision since no such "inheritances" provide a real implementation to mask.

Prior to typing, this was not widely used in Python, where the philosophy was Duck typing (try instead of check). Typing introduce the need for a `Protocol` mechanism, which can also be used to define something similar to an interface.


In [ ]:
from typing import Protocol, runtime_checkable


@runtime_checkable  # To be able to use isinstance at runtime
class SupportsFlush(Protocol):
    def flush(self) -> None: ...


class Serializer(SupportsFlush):
    def flush(self) -> None:
        print("Flushing data...")


When used a priori to build a class hierarchy, we inherit from the `Protocol` child class. `Protocol` can be used at posteriori as well without being part of the class hierarchy.

### Mixin

A common way to use multiple inheritance is to use the Mixin construct. A Mixin is a piece of code designed that encapsulate a common behavior, allowing the share the implementation between class that are not part of a same hierarchy. 

Here is an example:

In [ ]:
class ClonableMixin:
    def clone(self, deep: bool = False):
        import copy
        return copy.deepcopy(self) if deep else copy.copy(self)
    
class ReprMixin:
    def __repr__(self):
        return (
            f"{self.__class__.__name__}("
            + ", ".join(f"{k}={v!r}" for k, v in self.__dict__.items())
            + ")"
        )
    
class Shape:
    def area(self) -> float:
        raise NotImplementedError()
    
    
class Rectangle(ClonableMixin, ReprMixin, Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height

Rectangle(10, 20).clone()

Mixin is a powerful construct, provided some good practices are observed:
- a mixin scope should be as small as possible (one mixin = one purpose), prefer chaining orthogonal scopes;
- suffix the name with `Mixin` to indicate a clear purpose;
- no `__init__` (avoid issues highlighted above), as little state as possible;
- always place the mixins first in the MRO;
- don't sublcass mixins;
- avoid when non-trivial

> Although mixins should avoid statefulness and initialization, one way to circumvent this limitation is to use 

https://realpython.com/python-mixin/#hide-the-state-in-function-closures 

### Cooperation
- cooperative `__init__`
- chainable hooks (cooperative super)
- mergeable result (cooperative super)




## Metaclasses

## Composition over inheritance

## Dynamic object creation ?

## Closing words
